<a href="https://colab.research.google.com/github/DeepOde/tones2tunes/blob/main/models/Model_BI_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [ ]:
class model1(nn.module):
  def __init__(self,seq_len,unique_notes,dropout=0.3,output_emb=100,rnn_unit=128, dense_unit=64):
    super().__init__()
    # this indicates the vocabulary(unique notes in the whole input)
    self.num_embeddings=unique_notes+1
    # this indicates the vector representation size for each note
    self.embedding_dim=output_emb
    # No of hidden layers in GRU
    self.hidden_unit=rnn_unit
    # creating embedding layer
    self.embedding=nn.Embedding(self.num_embeddings, self.embedding_dim)
    # creating bidirectional GRU
    # by default pytorch returns all the hidden state repre's
    # for more read this [@ https://discuss.pytorch.org/t/pytorch-equivalent-to-keras-layers-lstm-return-sequences-false/53976/2]
    self.BiGRU1=nn.GRU(self.embedding_dim,self.hidden_unit,bidirectional=True)
    # have to implement self attention(Have to check)